# Poker Texas Hold'em Coding Dojo

In [9]:
from collections import Counter, defaultdict
from itertools import combinations

## The input
The input of the game (i.e. the hands of the Poker game) were hard-coded as variable: I preferred to focus on solving the problem rather than managing input lines from command line or file.
No checks on the input cards were made, supposing that the given input is well-formatted.

In [10]:
INPUT=[
  "Kc 9s Ks Kd 9d 3c 6d",
  "Kc Ts Ks Kd Td 3c 6d",
  "9c Ah Ks Kd 9d 3c 6d",
  "Ac Qc Ks Kd 9d 3c",
  "9h 5s",
  "4d 2d Ks Kd 9d 3c 6d",
  "7s Ts Ks Kd 9d"
]

## Utilities
I defined two dictionnaries:
- FACES_DICT: to have a sortable value for the card faces
- RANKS: to associate to each rank a sortable value and a human readable string

In [11]:
FACES_DICT={
    "2":2,
    "3":3,
    "4":4,
    "5":5,
    "6":6,
    "7":7,
    "8":8,
    "9":9,
    "T":10,
    "J":11,
    "Q":12,
    "K":13,
    "A":14
    }

RANKS={
 "R_FLUSH":{"value":10, "text":"Royal Flush"},
 "S_FLUSH":{"value":9, "text":"Straight Flush"},
 "FOUR":{"value":8,"text":"Four of a kind"},
 "FULL_H":{"value":7,"text":"Full House"},
 "FLUSH":{"value":6,"text":"Flush"},
 "STRAIGHT":{"value":5, "text":"Straight"},
 "THREE":{"value":4, "text":"Three of a kind"},
 "2_PAIR":{"value":3, "text":"Two pair"},
 "PAIR":{"value":2, "text":"Pair"},
 "HIGH":{"value":1, "text":"High card"},
 "FOLD":{"value":0, "text":"Fold"}
}

## The Python Classes
To solve the problem, I defined four main classes.

### Card
The Card class defines a card of the game, basically constituted by the suit, and the face, which gives the value of the card. 
Using the FACES_DICT dictionnary, an integer was associated to each face in order to be able to sort cards depending on their value.

In [12]:
class Card:
    def __init__(self,twocharstring):
        self.face=twocharstring[0]
        self.suit=twocharstring[1]
        self.sortable_face=FACES_DICT[self.face]
        
    def __str__(self):
        return(self.face+self.suit)
    
    def __repr__(self):
        return str(self)

### Hand
The Hand class defines the hand of a player. It is formed by 7 cards: the 2 own cards, plus the 5 community cards on the table. Only the best 5 of those will be used by the player, while the least 2 cards will be unused.
This class stores the collection of cards of the player, and a set of methods to assign the best possible rank for the given hand.
In particular, the "getBestRank" method generates all the possible 5-cards subset for the 7-cards hand, using the itertools.combination method.

In [ ]:
class Hand:
    def __init__(self,hand_array):
        self.cards=list(map(lambda x: Card(x),hand_array))
        self.best_rank="HIGH"
        self.best_combination=None,
        self.getBestRank()
        
    def assignBestRank(self,new_rank, comb):
        if RANKS[new_rank]["value"]>RANKS[self.best_rank]["value"]:
            self.best_rank=new_rank
            self.best_combination=comb
        
    def getPossibleCombinations(self):
        return combinations(self.cards,5)
    
    def getBestRank(self):
        if self.isFold():
            self.best_combination=Combination(self.cards,[])
            self.best_rank="FOLD"
        else:
            for _c in self.getPossibleCombinations():
                c=Combination(_c,list(set(self.cards)-set(_c)))
                c.getLocalBestRank()
                self.assignBestRank(c.best_rank,c)
            
    def isFold(self):
        return len(self.cards)<7


### Combination


In [13]:
class Combination:
    
    def __init__(self,combination, unused_cards):
        self.cards=list(combination),
        self.cards=self.cards[0]
        self.suits=list(map(lambda x: x.suit,self.cards))
        self.faces=list(map(lambda x: x.sortable_face,self.cards))
        self.counter=Counter(self.faces)
        self.best_rank="HIGH"
        self.ranked=[]
        self.kickers=[]
        self.unused=unused_cards

        
    def assignBestRank(self,new_rank):
        if RANKS[new_rank]["value"]>RANKS[self.best_rank]["value"]:
            self.best_rank=new_rank
            
    def isSameSuit(self):
        return self.suits.count(self.suits[0])==len(self.suits)
    
    def hasStraight(self):
        return sorted(self.faces)==list(range(min(self.faces), max(self.faces)+1))
    
    
    def getLocalBestRank(self):
        # same suits
        if self.isSameSuit():
            if self.hasStraight():
                if sorted(self.faces)==[10,11,12,13,14]:
                    self.assignBestRank("R_FLUSH")
                else:
                    self.assignBestRank("S_FLUSH")
            else:
                self.assignBestRank("FLUSH")
        # different suits
        else:
            if self.hasStraight():
                self.assignBestRank("STRAIGHT")
            else:
                if 4 in self.counter.values():
                    self.assignBestRank("FOUR")
                elif 3 in self.counter.values():
                   
                    if 2 in self.counter.values():
                        self.assignBestRank("FULL_H")
                    else:
                        self.assignBestRank("THREE")
                elif 2 in self.counter.values():
                    if list(self.counter.values()).count(2)==2:
                        self.assignBestRank("2_PAIR")
                    else:
                        self.assignBestRank("PAIR")
                else:
                    self.assignBestRank("HIGH")
                    
        self.defineRanked()
                    
    def defineRanked(self):
        if self.isSameSuit() or self.hasStraight():
            self.ranked=list(sorted(self.cards,key=lambda x:x.sortable_face,reverse=True))
        else:
            if self.best_rank=="HIGH":
                self.ranked.append(list(sorted(self.cards,key=lambda x:x.sortable_face,reverse=True))[0])
                
            else:
                self.ranked=list(sorted(filter(lambda x:self.counter[x.sortable_face]>1, self.cards),key=(lambda x:x.sortable_face), reverse=True))
                
        self.defineKickers()

                    
    def defineKickers(self):
        self.kickers=list(sorted(list(set(self.cards) - set(self.ranked)) + self.unused, key=(lambda x:x.sortable_face), reverse=True))
        self.unused=self.kickers[-2:]
        self.kickers=list(set(self.kickers) - set(self.unused))

In [15]:
class Game:
    
    def __init__(self, hands):
        self.hands=hands
        self.best_rank="FOLD"
        self.best_hands=[]
        
        
    def finalRanking(self):
        for h in hands:
            if h.best_rank!="FOLD":
                if RANKS[h.best_rank]["value"]>RANKS[self.best_rank]["value"]:
                    self.best_rank=h.best_rank
                    self.best_hands=[]
                    self.best_hands.append(h)
                elif RANKS[h.best_rank]["value"]<RANKS[self.best_rank]["value"]:
                    pass
                else:
                    # tie, compare ranked
                    ranked_a=list(map(lambda x:x.sortable_face,self.best_hands[0].best_combination.ranked))
                    ranked_b=list(map(lambda x:x.sortable_face,h.best_combination.ranked))
                    res=self.compareGroupedSequence(ranked_a, ranked_b)
                    if res==2:
                        self.best_rank=h.best_rank
                        self.best_hands=[]
                        self.best_hands.append(h)
                    elif res==1:
                        self.worst_hands.append(h)
                    else:
                        # tie, compare kickers
                        kickers_a=list(map(lambda x:x.sortable_face,self.best_hands[0].best_combination.kickers))
                        kickers_b=list(map(lambda x:x.sortable_face,h.best_combination.kickers))
                        if len(kickers_a)>0:
                            res2=self.compareGroupedSequence(kickers_a, kickers_b)
                            if res2==2:
                                self.best_rank=h.best_rank
                                self.best_hands=[]
                                self.best_hands.append(h)
                            elif res2==1:
                                pass
                            else:
                                self.best_hands.append(h)
                        else:
                            self.best_hands.append(h)
                    
                
    def compareSequence(self,a,b):
        for i in range(len(a)):
            if a[i]>b[i]:
                return 1
            elif a[i]<b[i]:
                return 2
            else:
                continue
        return 0
    
    
    def compareGroupedSequence(self,a,b):
        a_count=Counter(a)
        b_count=Counter(b)
        
        v = defaultdict(list)
        for key, value in sorted(a_count.items(),reverse=True):
            v[value].append(key)
        a_count=v
        
        v = defaultdict(list)
        for key, value in sorted(b_count.items(),reverse=True):
            v[value].append(key)
        b_count=v
        
        res=0
        for k in sorted(a_count.keys(), reverse=True):
            res=self.compareSequence(a_count[k], b_count[k])
            if res==0:
                continue
            else:
                return res
        return res

In [ ]:
hands=[]
for a in INPUT:
    hand=Hand(a.split(" "))
    hands.append(hand)

In [16]:
g=Game(hands)
g.finalRanking()
for h in g.hands:
    if h.best_rank!="FOLD":
       if h in g.best_hands:
           print (h.best_combination.ranked+h.best_combination.kickers+h.best_combination.unused, RANKS[h.best_rank]["text"], "(Winner)")
       else:
           print (h.best_combination.ranked+h.best_combination.kickers+h.best_combination.unused, RANKS[h.best_rank]["text"])    
    else:
        print (h.best_combination.cards)

[Kc, Ks, Kd, 9s, 9d, 6d, 3c] Full House
[Kc, Ks, Kd, Ts, Td, 6d, 3c] Full House (Winner)
[Ks, Kd, 9c, 9d, Ah, 6d, 3c] Two pair
[Ac, Qc, Ks, Kd, 9d, 3c]
[9h, 5s]
[Kd, 9d, 6d, 4d, 2d, Ks, 3c] Flush
[7s, Ts, Ks, Kd, 9d]
